# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
# encode 'homeowner' feature with LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

# Split the Data into Training and Testing

In [5]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)

# Create our target
y = df['loan_status']

In [6]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [13]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9520479254722232

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  563,    56],
       [  102, 18663]], dtype=int64)

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8466    0.9095    0.9946    0.8769    0.9511    0.8969       619
   low_risk     0.9970    0.9946    0.9095    0.9958    0.9511    0.9123     18765

avg / total     0.9922    0.9918    0.9122    0.9920    0.9511    0.9118     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled1, y_resampled1 = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled1)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [17]:
# Train the Logistic Regression model using the resampled data
model1 = LogisticRegression(solver='lbfgs', random_state=1)
model1.fit(X_resampled1, y_resampled1)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred1 = model1.predict(X_test)
balanced_accuracy_score(y_test, y_pred1)

0.9936781215845847

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred1)

array([[  615,     4],
       [  116, 18649]], dtype=int64)

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred1, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8413    0.9935    0.9938    0.9111    0.9937    0.9874       619
   low_risk     0.9998    0.9938    0.9935    0.9968    0.9937    0.9874     18765

avg / total     0.9947    0.9938    0.9935    0.9941    0.9937    0.9874     19384



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled2, y_resampled2 = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled2)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [24]:
# Train the Logistic Regression model using the resampled data
model2 = LogisticRegression(solver='lbfgs', random_state=1)
model2.fit(X_resampled2, y_resampled2)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
y_pred2 = model2.predict(X_test)
balanced_accuracy_score(y_test, y_pred2)

0.9936781215845847

In [26]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred2)

array([[  615,     4],
       [  116, 18649]], dtype=int64)

In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred2, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8413    0.9935    0.9938    0.9111    0.9937    0.9874       619
   low_risk     0.9998    0.9938    0.9935    0.9968    0.9937    0.9874     18765

avg / total     0.9947    0.9938    0.9935    0.9941    0.9937    0.9874     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled3, y_resampled3 = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled3)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [29]:
# Train the Logistic Regression model using the resampled data
model3 = LogisticRegression(solver='lbfgs', random_state=1)
model3.fit(X_resampled3, y_resampled3)

LogisticRegression(random_state=1)

In [30]:
# Calculate the balanced accuracy score
y_pred3 = model3.predict(X_test)
balanced_accuracy_score(y_test, y_pred3)

0.9865149130022852

In [31]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred3)

array([[  606,    13],
       [  112, 18653]], dtype=int64)

In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred3, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8440    0.9790    0.9940    0.9065    0.9865    0.9717       619
   low_risk     0.9993    0.9940    0.9790    0.9967    0.9865    0.9746     18765

avg / total     0.9943    0.9936    0.9795    0.9938    0.9865    0.9745     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled4, y_resampled4 = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled4)

Counter({'high_risk': 55609, 'low_risk': 55948})

In [34]:
# Train the Logistic Regression model using the resampled data
model4 = LogisticRegression(solver='lbfgs', random_state=1)
model4.fit(X_resampled4, y_resampled4)

LogisticRegression(random_state=1)

In [35]:
# Calculate the balanced accuracy score
y_pred4 = model4.predict(X_test)
balanced_accuracy_score(y_test, y_pred4)

0.9935182494822666

In [36]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred4)

array([[  615,     4],
       [  122, 18643]], dtype=int64)

In [37]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred4, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8345    0.9935    0.9935    0.9071    0.9935    0.9871       619
   low_risk     0.9998    0.9935    0.9935    0.9966    0.9935    0.9871     18765

avg / total     0.9945    0.9935    0.9935    0.9938    0.9935    0.9871     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   The two oversampling models have the best balanced accuracy score at 99.37%.

2. Which model had the best recall score?

    The two oversampling models have the best recall score at 99.38%.

3. Which model had the best geometric mean score?

   The two oversampling models have the best geometric mean score at 99.37%.
